# SSD / TextBoxes convertion (`pytorch -> tensorflow`)

In [1]:
import os
import torch
import importlib
import numpy as np
import tensorflow as tf

from utils import is_equal
from model import Textboxes, ResNet, SSD
from models.weights_converter import *
from custom_architectures import textboxes_arch

try:
    tf.config.set_visible_devices([], 'GPU')
except:
    pass

def load_model(model_name, ckpt_path, backbone = 'RN512', img_size = 512, truncate = False, device = 'cpu'):
    if "SSD" in model_name:
         model = SSD(model_name, truncate, backbone = ResNet(backbone), figsize = img_size, num_classes = 2)
    else:
        model = Textboxes(model_name, truncate, backbone = ResNet(backbone), figsize = img_size, num_classes = 2)

    checkpoint   = torch.load(ckpt_path, map_location = device)

    model_state_dict = {k.replace('module.', '') : v for k, v in checkpoint["model_state_dict"].items()}
    model.load_state_dict(model_state_dict)
    model.to(device)
    model.eval()
    
    return model

2023-07-25 13:22:39.230984: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-25 13:22:39.345461: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-25 13:22:39.369672: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. E

In [2]:
ckpt_path    = '../pytorch/SSD-TB/SSD512_R152.pth'
model_name   = 'SSD'

img_size  = 512
truncate  = False
backbone  = 'ResNet152'

assert os.path.exists(ckpt_path), 'Checkpoint does not exist !'

pretrained_model = load_model(
    model_name, ckpt_path, truncate = truncate, img_size = img_size, backbone = backbone, device = 'cpu'
)

/etinfo/users2/qlanglois/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/etinfo/users2/qlanglois/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
print(pretrained_model)

SSD(
  (feature_extractor): ResNet(
    (feature_extractor): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=T

## SSD convertion

### Full model convertion (SSD 512 - ResNet152 backbone)

In [6]:
importlib.reload(textboxes_arch)

pretrained = pretrained_model
model      = textboxes_arch.SSD((512, 512, 3), backbone = 'ResNet152')

_ = name_based_partial_transfer_learning(model, pretrained, skip_root = False)

/etinfo/users2/qlanglois/phd_text_detection/tensorflow/models/weights_converter.py:102: UserWarning: Unable to determine the root based on candidates : {'feature_extractor': 870, 'additional_blocks': 72, 'loc': 14, 'conf': 14}
  warnings.warn('Unable to determine the root based on candidates : {}'.format(parts))
/etinfo/users2/qlanglois/phd_text_detection/tensorflow/models/weights_converter.py:102: UserWarning: Unable to determine the root based on candidates : {'feature_extractor': 725, 'SSD': 88}
  warnings.warn('Unable to determine the root based on candidates : {}'.format(parts))


Weights transfered successfully !


In [7]:
for i in range(5):
    img_size = 512
    inp = np.random.normal(size = (1, img_size, img_size, 3))
    pt_inp = torch.FloatTensor(np.transpose(inp, [0, 3, 1, 2]))
    
    with torch.no_grad():
        pt_out = pretrained.eval()(pt_inp)

    with tf.device('cpu'):
        tf_out = model(inp, training = False)
        
    print(is_equal(pt_out, tf_out, max_err = 5e-3)[1])

Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !


### Feature Extractor convertion (`ResNet152`)

In [24]:
pretrained = pretrained_model.feature_extractor

print(pretrained)

ResNet(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Co

In [27]:
importlib.reload(textboxes_arch)

model = textboxes_arch.ResNet152((None, None, 3))

_ = name_based_partial_transfer_learning(model, pretrained, skip_root = False)

Weights transfered successfully !


In [30]:
for i in range(5):
    img_size = 128 + 32 * (i + 1)
    inp = np.random.normal(size = (i + 1, img_size, img_size, 3))
    pt_inp = torch.FloatTensor(np.transpose(inp, [0, 3, 1, 2]))
    
    with tf.device('cpu'):
        tf_out = model(inp, training = False)
    
    with torch.no_grad():
        pt_out = pretrained.eval()(pt_inp)
        pt_out = np.transpose(pt_out.numpy(), [0, 2, 3, 1])
    
    print(is_equal(pt_out, tf_out, max_err = 1e-3)[1])

Value are equals !
Value are equals !
Value are equals !
Values differ (2 / 4194304 diff, 0.000%) : max 0.0011135339736938477 - mean 1.755756966304034e-05 - min 0.0
Values differ (1 / 6635520 diff, 0.000%) : max 0.0010211467742919922 - mean 1.7843851310317405e-05 - min 0.0


### Bottleneck convertion

In [3]:
pretrained = pretrained_model.feature_extractor.feature_extractor[4][0]

print(pretrained)

Bottleneck(
  (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (downsample): Sequential(
    (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)


In [7]:
importlib.reload(textboxes_arch)

model      = textboxes_arch.Bottleneck((None, None, 64), filters = 64)

_ = name_based_partial_transfer_learning(model, pretrained)

Weights transfered successfully !


/etinfo/users2/qlanglois/phd_text_detection/tensorflow/models/weights_converter.py:102: UserWarning: Unable to determine the root based on candidates : {'conv1': 1, 'bn1': 5, 'conv2': 1, 'bn2': 5, 'conv3': 1, 'bn3': 5, 'downsample': 6}
  warnings.warn('Unable to determine the root based on candidates : {}'.format(parts))


In [8]:
for i in range(5):
    img_size = 32 * (i + 1)
    inp = np.random.normal(size = (i + 1, img_size, img_size, 64))
    pt_inp = torch.FloatTensor(np.transpose(inp, [0, 3, 1, 2]))
    
    with tf.device('cpu'):
        tf_out = model(inp, training = False)
    
    with torch.no_grad():
        pt_out = pretrained.eval()(pt_inp).numpy()
        pt_out = np.transpose(pt_out, [0, 2, 3, 1])
    
    print(is_equal(pt_out, tf_out, max_err = 1e-4)[1])

Value are equals !
Value are equals !
Value are equals !
Value are equals !
Value are equals !
